# 杭州萧山区项目`CMAQ-ISAM`
## 使用`M3Tools`生成的`IOAPI`接口的`GRIDMASK.nc`
## 导出ASCII数据

---
*@author: Evan*\
*@date: 2023-06-29*

In [1]:
import xarray as xr
import numpy as np

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
grid = xr.open_dataset('./input/GRIDCRO2D.nc')
lat = grid.LAT.squeeze()
lon = grid.LON.squeeze()

In [3]:
import geopandas as gpd
import shapely.geometry as sgeom
from shapely.prepared import prep

def polygon_to_mask(polygon, x, y):
    '''
    Generate a mask array of points falling into the polygon
    '''
    x = np.atleast_1d(x)
    y = np.atleast_1d(y)
    mask = np.zeros(x.shape, dtype=bool)

    # if each point falls into a polygon, without boundaries
    prepared = prep(polygon)
    for index in np.ndindex(x.shape):
        point = sgeom.Point(x[index], y[index])
        if prepared.contains(point):
            mask[index] = True

    return mask

In [6]:
ds = xr.Dataset(
    data_vars=dict(
        xiaoshan   = (['ROW','COL'],np.zeros_like(lat)),
        north = (['ROW','COL'],np.zeros_like(lat)),
        northwest   = (['ROW','COL'],np.zeros_like(lat)),
        southeast     = (['ROW','COL'],np.zeros_like(lat)),
        fuyang     = (['ROW','COL'],np.zeros_like(lat)),
        shaoxing   = (['ROW','COL'],np.zeros_like(lat)),
        hangzhou   = (['ROW','COL'],np.zeros_like(lat)),
    ),
    coords=dict(
        ROW = grid.ROW.data,
        COL = grid.COL.data,
    )
)
ds

<xarray.Dataset>
Dimensions:    (ROW: 138, COL: 135)
Coordinates:
  * ROW        (ROW) int64 0 1 2 3 4 5 6 7 8 ... 130 131 132 133 134 135 136 137
  * COL        (COL) int64 0 1 2 3 4 5 6 7 8 ... 127 128 129 130 131 132 133 134
Data variables:
    xiaoshan   (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    north      (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    northwest  (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    southeast  (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    fuyang     (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    shaoxing   (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    hangzhou   (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [7]:
regions = ['xiaoshan', 'north', 'northwest', 'southeast', 'fuyang', 'shaoxing', 'hangzhou',]

for region in regions:
    shp = gpd.read_file(f'./shp/{region}.shp')
    for i in range(np.size(lat,0)):
        for j in range(np.size(lat,1)):
            if polygon_to_mask(shp.geometry[0], lon[i, j], lat[i, j]) == False:
                ds[f'{region}'][i, j] = 0
            else:
                ds[f'{region}'][i, j] = 1

In [8]:
df = ds.to_dataframe()
# m3mask读取的顺序是先COL后ROW
df = df.swaplevel('ROW','COL')

# 将数据设置为int
df = df.astype(int)
df

,,xiaoshan,north,northwest,southeast,fuyang,shaoxing,hangzhou
COL,ROW,,,,,,,
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
130,137,0,0,0,0,0,0,0
131,137,0,0,0,0,0,0,0
132,137,0,0,0,0,0,0,0


In [9]:
# fortran是从1开始的，将COL和ROW的数值+1
df = df.rename(index=lambda x: x + 1, level=0)
df = df.rename(index=lambda x: x + 1, level=1)
df

,,xiaoshan,north,northwest,southeast,fuyang,shaoxing,hangzhou
COL,ROW,,,,,,,
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
131,138,0,0,0,0,0,0,0
132,138,0,0,0,0,0,0,0
133,138,0,0,0,0,0,0,0


In [10]:
for region in regions:
    df_region = df[region]
    df_region = df_region[(df_region != 0)]
    df_region.to_csv(f'./output/df_{region}.csv',header=False)

旧地区-放弃

In [ ]:
ds = xr.Dataset(
    data_vars=dict(
        xiaoshan   = (['ROW','COL'],np.zeros_like(lat)),
        shangcheng = (['ROW','COL'],np.zeros_like(lat)),
        qiantang   = (['ROW','COL'],np.zeros_like(lat)),
        yuhang     = (['ROW','COL'],np.zeros_like(lat)),
        fuyang     = (['ROW','COL'],np.zeros_like(lat)),
        shaoxing   = (['ROW','COL'],np.zeros_like(lat)),
        ningbo     = (['ROW','COL'],np.zeros_like(lat)),
        shanghai   = (['ROW','COL'],np.zeros_like(lat)),
        suzhou     = (['ROW','COL'],np.zeros_like(lat)),
        wuxi       = (['ROW','COL'],np.zeros_like(lat)),
        xuancheng  = (['ROW','COL'],np.zeros_like(lat)),
        huzhou     = (['ROW','COL'],np.zeros_like(lat)),
        jiaxing    = (['ROW','COL'],np.zeros_like(lat)),
        hangzhou   = (['ROW','COL'],np.zeros_like(lat)),
        huangshan  = (['ROW','COL'],np.zeros_like(lat)),
        jinhua     = (['ROW','COL'],np.zeros_like(lat)),
        quzhou     = (['ROW','COL'],np.zeros_like(lat)),
    ),
    coords=dict(
        ROW = grid.ROW.data,
        COL = grid.COL.data,
    )
)
ds

<xarray.Dataset>
Dimensions:          (ROW: 138, COL: 135)
Coordinates:
  * ROW              (ROW) int64 0 1 2 3 4 5 6 7 ... 131 132 133 134 135 136 137
  * COL              (COL) int64 0 1 2 3 4 5 6 7 ... 128 129 130 131 132 133 134
Data variables: (12/17)
    xiaoshan         (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    shangcheng_etal  (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    qiantang         (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    yuhang           (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    fuyang           (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    shaoxing         (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...               ...
    huzhou           (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    jiaxing          (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    hangzhou         (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    huangshan        (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    jinhua           (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    quzhou           (ROW, COL) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [ ]:
regions = ['xiaoshan', 'shangcheng', 'qiantang', 'yuhang', 'fuyang',
           'shaoxing', 'ningbo', 'shanghai', 'suzhou', 'wuxi',
           'xuancheng', 'huzhou', 'jiaxing', 'hangzhou', 'huangshan',
           'jinhua', 'quzhou',]

for region in regions:
    shp = gpd.read_file(f'./shp/{region}.shp')
    for i in range(np.size(lat,0)):
        for j in range(np.size(lat,1)):
            if polygon_to_mask(shp.geometry[0], lon[i, j], lat[i, j]) == False:
                ds[f'{region}'][i, j] = 0
            else:
                ds[f'{region}'][i, j] = 1